In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pybedtools import BedTool
from tqdm.notebook import tqdm


def get_logger(level="INFO"):
    from loguru import logger

    try:
        logger.remove(handler_id=0)
        logger.add(sink=sys.stdout, level=level, colorize=True)
    except ValueError:
        pass

    return logger


def set_notebook_options():
    import matplotlib.pyplot as plt
    import pandas as pd
    from matplotlib_inline.backend_inline import set_matplotlib_formats

    plt.matplotlib.rcParams["figure.dpi"] = 210
    set_matplotlib_formats("retina")

    pd.set_option("display.max_columns", 100)


set_notebook_options()
logger = get_logger()

In [2]:
def get_sharpr_score_fn(cell_type, promoter):
    assert promoter in ["minP", "SV40P"]

    if cell_type.lower() == "hepg2":
        return f"data/raw/HEPG2_SHARPR-MPRA_scores/basepredictions_HEPG2_ScaleUpDesign1and2_{promoter}.txt"
    elif cell_type.lower() == "k562":
        return f"data/raw/K562_SHARPR-MPRA_scores/basepredictions_K562_ScaleUpDesign1and2_{promoter}.txt"
    else:
        raise ValueError()


def compute_sharpr_scores(cell_type, promoter):
    sharpr_score_fn = get_sharpr_score_fn(cell_type, promoter)

    sharpr_scores = pd.read_csv(
        sharpr_score_fn,
        sep="\t",
        names=["region_id"] + list(range(295)),
        index_col=0,
    )

    sharpr_score_df = sharpr_scores.iloc[:, ::5].copy()
    sharpr_score_df.columns = range(sharpr_score_df.shape[1])
    return sharpr_score_df


def load_logratios(cell_type, promoter, normalize=True):
    # dna threshold = 20
    logratio_df_1 = pd.read_csv(
        f"data/processed/sharpr_20/logratios.{cell_type}.{promoter}.Rep1.tsv",
        sep="\t",
        index_col=0,
    )

    logratio_df_2 = pd.read_csv(
        f"data/processed/sharpr_20/logratios.{cell_type}.{promoter}.Rep2.tsv",
        sep="\t",
        index_col=0,
    )

    logratio_df = (logratio_df_1 + logratio_df_2) / 2
    logratio_df.columns = logratio_df.columns.map(int)

    if normalize:
        logratio_df = (
            logratio_df - logratio_df.mean().mean()
        ) / logratio_df.stack().std()

    return logratio_df


def load_dragonn(cell_type, promoter, seq_model="DeepFactorizedModel"):
    if seq_model == "DeepFactorizedModel":
        fn = f"data/processed/dragonn/features.dragoNN_DeepFactorizedModel.{cell_type}.{promoter}.pooled.tsv.gz"
    else:
        raise ValueError()

    dragonn_prediction_df = pd.read_csv(
        fn,
        sep="\t",
        index_col=0,
    )

    dragonn_prediction_df.columns = dragonn_prediction_df.columns.map(int)

    # standardize
    dragonn_prediction_df = (
        dragonn_prediction_df - dragonn_prediction_df.mean().mean()
    ) / dragonn_prediction_df.stack().std()

    return dragonn_prediction_df


def get_design_matrix():
    num_A = 59

    X_ = np.zeros(shape=[num_A, 31])

    for i in range(num_A):
        start_ = i - 28 if i - 28 > 0 else 0
        end_ = i + 1

        X_[i, start_:end_] = 1

    X = pd.DataFrame(X_).T

    return X

In [3]:
def calculate_scores(data_df, alpha_=100):
    from sklearn.linear_model import Ridge

    regions = data_df.index

    X = get_design_matrix()

    score_d = {}
    for region_id in tqdm(regions):
        y = (
            data_df.loc[region_id, :]
            .interpolate(method="linear", limit=5)
            .bfill(limit=5)
            .ffill(limit=5)
        )

        if y.isna().any():
            logger.debug(f"Region failed: {region_id=}")
            continue

        model = Ridge(alpha=alpha_, fit_intercept=False)
        model.fit(X, y)

        score = pd.Series(model.coef_, name=region_id)
        score_d[region_id] = score

    score_df = pd.DataFrame(score_d).T

    return score_df

In [4]:
def process_scores(
    cell_type, promoter, alpha_tiling, alpha_dragonn, seq_model="DeepFactorizedModel"
):
    logratio_df = load_logratios(cell_type, promoter)

    tiling_score_df = calculate_scores(logratio_df, alpha_=alpha_tiling)

    dragonn_prediction_df = load_dragonn(cell_type, promoter, seq_model=seq_model)

    dragonn_score_df = calculate_scores(dragonn_prediction_df, alpha_=alpha_dragonn)

    sharpr_score_df = compute_sharpr_scores(cell_type, promoter)

    # normalize scores
    tiling_score_df = (
        tiling_score_df - tiling_score_df.mean().mean()
    ) / tiling_score_df.stack().std()

    dragonn_score_df = (
        dragonn_score_df - dragonn_score_df.mean().mean()
    ) / dragonn_score_df.stack().std()

    sharpr_score_df = (
        sharpr_score_df - sharpr_score_df.mean().mean()
    ) / sharpr_score_df.stack().std()

    # serialize scores to parquet
    tiling_score_df.to_parquet(
        f"data/processed/01_scores/tiling_scores.{cell_type}.{promoter}.alpha_{alpha_tiling}.norm.parquet"
    )

    dragonn_score_df.to_parquet(
        f"data/processed/01_scores/dragonn_scores.{cell_type}.{promoter}.alpha_{alpha_dragonn}.norm.parquet"
    )

    sharpr_score_df.to_parquet(
        f"data/processed/01_scores/sharpr_scores.{cell_type}.{promoter}.norm.parquet"
    )


cell_types = ["HepG2", "K562"]
promoters = ["minP", "SV40P"]

alpha_tilings = [1000]
alpha_dragonns = [10]

for cell_type in cell_types:
    for promoter in promoters:
        for alpha_tiling in alpha_tilings:
            for alpha_dragonn in alpha_dragonns:
                logger.info(
                    f"Processing {cell_type=}, {promoter=}, {alpha_tiling=}, {alpha_dragonn=}"
                )
                process_scores(cell_type, promoter, alpha_tiling, alpha_dragonn)